In [ ]:
import numpy as np
import sympy as sym
from pybullet_suite import *
from .parse_urdf import *

In [ ]:
# calculate forward kinematics
def get_T_dict(link_dict, joint_dict, qs):
    T_dict = {}
    link_name = "panda_grasptarget"
    while True:
        if "parent_joint" not in link_dict[link_name]:  break
        parent_joint_name = link_dict[link_name]["parent_joint"]
        parent_joint = joint_dict[parent_joint_name]
        T_dict[parent_joint_name] = generate_sym_matrix(parent_joint["origin_rpy"], parent_joint["origin_xyz"])
        link_name = joint_dict[parent_joint_name]["parent"]

    i = 0
    while True:
        if "joint" not in link_dict[link_name]: break
        joint_name = link_dict[link_name]["joint"]
        if joint_dict[joint_name]["joint_type"] == "revolute":
            T_dict[f"q_{joint_name}"] = yaw_rot_mat(qs[i])
        link_name = joint_dict[joint_name]["child"]
        i += 1
    return T_dict

#sympy
def generate_sym_matrix(rpy=[0,0,0], trans=[0,0,0]):
    rot_mat = Rotation.from_euler("zyx", rpy[::-1]).as_matrix()
    rot_mat[np.abs(rot_mat) < 1e-9] = 0.
    T = np.block([[rot_mat, np.array(trans)[:,None]], [0,0,0,1]])
    return sym.Matrix(T)

def yaw_rot_mat(q):
    s, c = sym.sin(q), sym.cos(q)
    return sym.Matrix([[c, -s, 0, 0],
                       [s, c, 0, 0],
                       [0, 0, 1, 0],
                       [0,0,0,1]])


def get_fk_sym_expr(link_dict, joint_dict, T_dict, root_link_name):
    link_name = root_link_name #"panda_link0"
    fk = sym.Matrix(np.eye(4))
    while True:
        if "joint" not in link_dict[link_name]: break
        joint_name = link_dict[link_name]["joint"]
        if joint_name in T_dict:
            fk *= T_dict[joint_name]
        if "q_"+joint_name in T_dict:
            fk *= T_dict["q_"+joint_name]
        link_name = joint_dict[joint_name]["child"]
    return fk

In [ ]:
# Define the path to the URDF file
urdf_file = "panda.urdf"

# Parse the URDF file using ElementTree
link_dict, joint_dict = get_link_joint_dict(urdf_file)

In [ ]:
q1, q2, q3, q4, q5, q6, q7 = sym.symbols('q1 q2 q3 q4 q5 q6 q7')
qs = [q1, q2, q3, q4, q5, q6, q7]
T_dict = get_T_dict(link_dict, joint_dict, qs)
fk = get_fk_sym_expr(link_dict, joint_dict, T_dict, "panda_link0")

In [ ]:
# test
q = panda.get_random_arm_angles()
q_values = {q:qval for q, qval in zip(qs, q)}
T_evaluated = fk.subs(q_values)

In [ ]:
world = BulletWorld(gui=False)
panda = world.load_robot(Panda, "panda")